Imports

In [15]:
import os
import torch
import torchvision
from torchvision import transforms, datasets
from torchvision.models import efficientnet_b0
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


Paths

In [16]:
data_dir = r"C:\Users\SHAHZOR AHMED\OneDrive\Desktop\Major project\A new approach\image_classification_dataset"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")


Transforms

In [17]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


Load Data

In [18]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)  # Should print: ['dent', 'no_damage', 'scratch']


Classes: ['dent', 'no_damage', 'scratch']


Model Setup

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = efficientnet_b0(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(num_ftrs, 3)
)
model = model.to(device)


Using device: cuda


c:\Users\SHAHZOR AHMED\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\SHAHZOR AHMED\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Weighted Loss + Optimizer + Scheduler

In [20]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Class counts for dent, no_damage, scratch
class_counts = [71, 404, 80]
class_labels = np.array([0, 1, 2])  #  convert to np.ndarray

# Compute balanced class weights
weights = compute_class_weight(class_weight='balanced',
                                classes=class_labels,
                                y=[
    *([0] * class_counts[0]),
    *([1] * class_counts[1]),
    *([2] * class_counts[2])
])

# Convert to tensor and move to device
weights = torch.tensor(weights, dtype=torch.float).to(device)

# Define loss with class weights
criterion = nn.CrossEntropyLoss(weight=weights)

# Optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)


Training Loop with Validation + Early Stopping

In [21]:
epochs = 15
best_val_acc = 0
early_stop_count = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    scheduler.step()

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{epochs} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        early_stop_count = 0
        torch.save(model.state_dict(), os.path.join(data_dir, "best_model.pth"))
    else:
        early_stop_count += 1
        if early_stop_count >= 5:
            print("Early stopping triggered.")
            break


Epoch 1/15 | Train Acc: 41.44% | Val Acc: 52.52%
Epoch 2/15 | Train Acc: 55.32% | Val Acc: 51.80%
Epoch 3/15 | Train Acc: 61.26% | Val Acc: 61.87%
Epoch 4/15 | Train Acc: 70.63% | Val Acc: 58.99%
Epoch 5/15 | Train Acc: 81.62% | Val Acc: 59.71%
Epoch 6/15 | Train Acc: 85.05% | Val Acc: 62.59%
Epoch 7/15 | Train Acc: 89.73% | Val Acc: 58.99%
Epoch 8/15 | Train Acc: 94.23% | Val Acc: 58.27%
Epoch 9/15 | Train Acc: 95.14% | Val Acc: 66.91%
Epoch 10/15 | Train Acc: 96.22% | Val Acc: 66.91%
Epoch 11/15 | Train Acc: 96.94% | Val Acc: 67.63%
Epoch 12/15 | Train Acc: 96.58% | Val Acc: 68.35%
Epoch 13/15 | Train Acc: 97.30% | Val Acc: 66.91%
Epoch 14/15 | Train Acc: 97.48% | Val Acc: 66.19%
Epoch 15/15 | Train Acc: 98.92% | Val Acc: 66.91%
